In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 112.0.5615
Get LATEST chromedriver version for 112.0.5615 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/112.0.5615.49/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\liz14\.wdm\drivers\chromedriver\win32\112.0.5615.49]


In [2]:
url = "https://www.imdb.com/list/ls006444168/"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [54]:
# Webscrape imdb movies from books list. 
movies_list = soup.find_all('div', class_='lister-list')

# Pull movie title, year released, movie duration, genre, movie rating ("certificate"), star rating, gross amount.

title = movies_list[0].find('h3').find('a').text

year = movies_list[0].find('span',class_='lister-item-year').text

movie_length = movies_list[0].find('span',class_='runtime').text

genre = movies_list[0].find('span',class_='genre').text

# javascript for certificate: # <span class="certificate">R</span>
certificate = movies_list[0].find('span',class_='certificate').text

# javascript for rating: <span class="ipl-rating-star__rating">7.5</span>
stars = movies_list[0].find('span',class_='ipl-rating-star__rating').text

# javascript for movie gross:
# <span class="text-muted">Gross:</span>
# <span name="nv" data-value="18,335,230">$18.34M</span>

# chatGPT   
gross = movies_list[0].find('span', string='Gross:')
gross_value = gross.find_next_sibling('span')['data-value']

#********************Discard later?
# # find the <p> tag with class 'text_muted text-small'
##p_tag = soup.find('p', class_='text-muted text-small')
#p_tag = soup.find('span', class_='text-muted').text
#p_tag = soup.find('span', name_='nv')


# # find the <span> tag with name 'nv' inside the <p> tag
#span_tag = p_tag.find('span', name='nv')

# # extract the text content of the <span> tag.
# text = span_tag.text.strip()

In [57]:
print(gross_value)

18,335,230


In [ ]:
movies_list

In [65]:
# Pull movie title, year released, movie duration, genre, movie rating ("certificate"), star rating, gross amount.

# Create dictionary of lists to hold....

movies_fr_books_dict = {}
movie_title =[]
year = []
movie_length =[]
movie_genre=[]
movie_rating=[]
#movie_star_rating=[]
#gross_amount=[]

def get_movietitle():
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    movies_list = soup.find_all('div', class_='lister-item-content')
    
    for movie in movies_list:
        title = movie.find('h3').find('a').text
        movie_year = movie.find('span', class_='lister-item-year').text
        movie_len = movie.find('span', class_='runtime').text
        genre = movie.find('span', class_='genre').text.strip()

# error starts from line 25  error error error error error error error error error error error error error line 25 wrong      
        rating = movie.find('span', class_='certificate').text.strip()
#certificate = movies_list[0].find('span',class_='certificate').text

        # recording values
        movie_title.append(title)
        year.append(movie_year)
        movie_length.append(movie_len)
        movie_genre.append(genre)
        #movie_rating.append(rating)

get_movietitle()

AttributeError: 'NoneType' object has no attribute 'text'

In [63]:
movies_fr_books_dict['movie_title'] = movie_title
movies_fr_books_dict['year'] = year
movies_fr_books_dict['movie_length'] = movie_length
movies_fr_books_dict['movie_genre'] = movie_genre

#movies_fr_books_dict['movie_rating'] = movie_rating

In [64]:
import pandas as pd
moviesfrombooks_data = pd.DataFrame(movies_fr_books_dict)
moviesfrombooks_data

,movie_title,year,movie_length,movie_genre
0,127 Hours,(2010),94 min,"Biography, Drama"
1,100 Rifles,(1969),110 min,"Adventure, Drama, War"
2,The Hunger Games,(2012),142 min,"Action, Adventure, Sci-Fi"
3,The Hunger Games: Catching Fire,(2013),146 min,"Action, Adventure, Sci-Fi"
4,A Beautiful Mind,(2001),135 min,"Biography, Drama"
...,...,...,...,...
295,Like Dandelion Dust,(2009),100 min,Drama
296,Public Enemies,(2009),140 min,"Action, Biography, Crime"
297,Confessions of a Shopaholic,(2009),104 min,"Comedy, Romance"
298,Inkheart,(2008),106 min,"Adventure, Family, Fantasy"


In [8]:
# need to update csv location since I moved imdb_movies_from_books.ipynb out of Resources folder.
moviesfrombooks_data.to_csv('imdb_movies_from_books.csv', index=False)

In [66]:
browser.quit()